In [1]:
import time,json,os
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from common import preprocess
from sklearn.svm import SVC
import joblib
from warnings import filterwarnings
filterwarnings("ignore") 
np.set_printoptions(suppress=True)
pd.set_option('display.float_format',lambda x : '%.4f' % x)

# check distribution

In [3]:
df_s0 = pd.read_csv("/data/paper/base/s-0.csv",index_col=0)
df_s1 = pd.read_csv("/data/paper/base/s-1.csv",index_col=0)
df_s2 = pd.read_csv("/data/paper/base/s-2.csv",index_col=0)
df_s3 = pd.read_csv("/data/paper/base/s-3.csv",index_col=0)

In [4]:
print(df_s0['label'].value_counts())
print(df_s1['label'].value_counts())
print(df_s2['label'].value_counts())
print(df_s3['label'].value_counts())

benign      325155
abnormal    174845
Name: label, dtype: int64
abnormal    55874950
benign       9187041
Name: label, dtype: int64
abnormal    32533580
benign      32528411
Name: label, dtype: int64
benign      43378034
abnormal    21683957
Name: label, dtype: int64


In [2]:
df1 = pd.read_csv("/data/paper/sample/s-0-1-0.csv",index_col=0)
df2 = pd.read_csv("/data/paper/sample/s-1-1-0.csv",index_col=0)
df3 = pd.read_csv("/data/paper/sample/s-2-1-0.csv",index_col=0)
df4 = pd.read_csv("/data/paper/sample/s-3-1-0.csv",index_col=0)

In [3]:
print(df1['label'].value_counts())
print(df2['label'].value_counts())
print(df3['label'].value_counts())
print(df4['label'].value_counts())

benign      185997
abnormal     64003
Name: label, dtype: int64
abnormal    241187
benign        8813
Name: label, dtype: int64
benign      125329
abnormal    124671
Name: label, dtype: int64
benign      191870
abnormal     58130
Name: label, dtype: int64


In [13]:
df1

,src_port,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,...,service_gssapi,service_http,service_irc,service_smb,service_smb;gssapi,service_ssh,service_ssl,proto_icmp,proto_tcp,proto_udp
244353,42100,7878,tcp,-,0.0000,0,0,OTH,0,0,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000
271526,60237,15600,udp,-,0.0000,0,0,S0,0,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
212100,34407,15600,udp,-,0.0000,0,0,S0,0,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
449950,5353,5353,udp,dns,13.0451,448,0,S0,0,4,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
316521,50948,9000,tcp,-,0.0000,0,0,S0,0,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375580,51732,15600,udp,-,0.0000,0,0,S0,0,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
267586,59336,80,tcp,http,139.6726,2422,3387,RSTR,0,10,...,0.0000,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000,0.0000
379311,51511,15600,udp,-,0.0000,0,0,S0,0,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
111239,47652,53,udp,dns,0.1276,55,223,SF,0,1,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000


In [52]:
import scipy
def quadratic_forms_matrix_euclidean(h1, h2):
    r"""
    Compute the bin-similarity matrix for the quadratic form distance measure.
    The matric :math:`A` for two histograms :math:`H` and :math:`H'` of size :math:`m` and
    :math:`n` respectively is defined as
    
    .. math::
    
        A_{m,n} = 1 - \frac{d_2(H_m, {H'}_n)}{d_{max}}
    
    with
    
    .. math::
    
       d_{max} = \max_{m,n}d_2(H_m, {H'}_n)
    
    See also
    --------
    quadratic_forms
    """
    A = scipy.repeat(h2[:,scipy.newaxis], h1.size, 1) # repeat second array to form a matrix
    A = scipy.absolute(A - h1) # euclidean distances
    return 1 - (A / float(A.max()))

#h1 = np.array(df1['src_bytes'].to_list())
#h2 = np.array(df1['src_bytes'].to_list())
h1 = np.array([1,2,34])
h2 = np.array([1,2,34])
h3 = np.array(df1['src_bytes'].head(10000).to_list())
h4 = np.array(df2['src_bytes'].head(10000).to_list())

print(h1.shape,h2.shape)
print(h3.shape,h4.shape)
quadratic_forms_matrix_euclidean(np.array(h1),np.array(h2))
quadratic_forms_matrix_euclidean(np.array(h3),np.array(h4))

(3,) (3,)
(10000,) (10000,)


array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [65]:
def get_mahalanobis(x, i, j):
    xT = x.T  # 求转置
    D = np.cov(xT)  # 求协方差矩阵
    invD = np.linalg.inv(D)  # 协方差逆矩阵
    assert 0 <= i < x.shape[0], "点 1 索引超出样本范围。"
    assert -1 <= j < x.shape[0], "点 2 索引超出样本范围。"
    x_A = x[i]
    x_B = x.mean(axis=0) if j == -1 else x[j]
    tp = x_A - x_B
    return np.sqrt(np.dot(np.dot(tp, invD), tp.T))
    
# 初始化数据点集，或者从其它地方加载
x = np.array([[3, 4], [5, 6], [2, 2], [8, 4]])
# 求第0个点到均值之间的马氏距离（j为-1时代表均值）
print(get_mahalanobis(x, 0, -1))
# 求第0个点到第1个点之间的马氏距离
print(get_mahalanobis(x, 0, 1))
# 求第2个点到第3个点之间的马氏距离（索引从0开始算起）
print(get_mahalanobis(x, 2, 3))

0.6396021490668313
1.243163121016122
2.2763607319179844


In [71]:
from scipy.spatial import distance
import random
 
array_1 = [-0.10577646642923355, 0.09617947787046432, 0.029290344566106796, 0.02092641592025757, -0.021434104070067406, -0.13410840928554535, 0.028282659128308296, -0.12082239985466003, 0.21936850249767303, -0.06512433290481567, 0.16812698543071747, -0.03302834928035736, -0.18088334798812866, -0.04598559811711311, -0.014739632606506348, 0.06391328573226929, -0.15650317072868347, -0.13678401708602905, 0.01166679710149765, -0.13967938721179962, 0.14632365107536316, 0.025218486785888672, 0.046839646995067596, 0.09690812975168228, -0.13414686918258667, -0.2883925437927246, -0.1435326784849167, -0.17896348237991333, 0.10746842622756958, -0.09142691642045975, 0.04860316216945648, 0.031577128916978836, -0.17280976474285126, -0.059613555669784546, -0.05718057602643967, 0.0401446670293808, 0.026440180838108063, -0.017025159671902657, 0.22091664373874664, 0.024703698232769966, -0.15607595443725586, -0.0018572667613625526, -0.037675946950912476, 0.3210170865058899, 0.10884962230920792, 0.030370134860277176, 0.056784629821777344, -0.030112050473690033, 0.023124486207962036, -0.1449904441833496, 0.08885903656482697, 0.17527811229228973, 0.08804896473884583, 0.038310401141643524, -0.01704210229218006, -0.17355971038341522, -0.018237406387925148, 0.030551932752132416, -0.23085585236549377, 0.13475817441940308, 0.16338199377059937, -0.06968289613723755, -0.04330683499574661, 0.04434924200177193, 0.22637797892093658, 0.07463733851909637, -0.15070196986198425, -0.07500549405813217, 0.10863590240478516, -0.22288714349269867, 0.0010778247378766537, 0.057608842849731445, -0.12828609347343445, -0.17236559092998505, -0.23064571619033813, 0.09910193085670471, 0.46647992730140686, 0.0634111613035202, -0.13985536992549896, 0.052741192281246185, -0.1558966338634491, 0.022585246711969376, 0.10514408349990845, 0.11794176697731018, -0.06241249293088913, 0.06389056891202927, -0.14145469665527344, 0.060088545083999634, 0.09667345881462097, -0.004665130749344826, -0.07927791774272919, 0.21978208422660828, -0.0016187895089387894, 0.04876316711306572, 0.03137822449207306, 0.08962501585483551, -0.09108036011457443, -0.01795950159430504, -0.04094596579670906, 0.03533276170492172, 0.01394269522279501, -0.08244197070598602, -0.05095399543642998, 0.04305890575051308, -0.1195211187005043, 0.16731074452400208, 0.03894471749663353, -0.0222858227789402, -0.07944411784410477, 0.0614166259765625, -0.1481470763683319, -0.09113290905952454, 0.14758692681789398, -0.24051085114479065, 0.164126917719841, 0.1753545105457306, -0.003193420823663473, 0.20875433087348938, 0.03357946127653122, 0.1259773075580597, -0.00022807717323303223, -0.039092566817998886, -0.13582147657871246, -0.01937306858599186, 0.015938198193907738, 0.00787206832319498, 0.05792934447526932, 0.03294186294078827]
array_2 = [-0.1966051608324051, 0.0940953716635704, -0.0031937970779836178, -0.03691547363996506, -0.07240629941225052, -0.07114037871360779, -0.07133384048938751, -0.1283963918685913, 0.15377545356750488, -0.091400146484375, 0.10803385823965073, -0.09235749393701553, -0.1866973638534546, -0.021168243139982224, -0.09094691276550293, 0.07300164550542831, -0.20971564948558807, -0.1847742646932602, -0.009817334823310375, -0.05971141159534454, 0.09904412180185318, 0.0278592761605978, -0.012554554268717766, 0.09818517416715622, -0.1747943013906479, -0.31632938981056213, -0.0864541232585907, -0.13249783217906952, 0.002135572023689747, -0.04935726895928383, 0.010047778487205505, 0.04549024999141693, -0.26334646344184875, -0.05263081565499306, -0.013573898002505302, 0.2042253464460373, 0.06646320968866348, 0.08540669083595276, 0.12267164140939713, -0.018634958192706108, -0.19135263562202454, 0.01208433136343956, 0.09216200560331345, 0.2779296934604645, 0.1531585156917572, 0.10681629925966263, -0.021275708451867104, -0.059720948338508606, 0.06610126793384552, -0.21058350801467896, 0.005440462380647659, 0.18833838403224945, 0.08883830159902573, 0.025969548150897026, 0.0337764173746109, -0.1585341989994049, 0.02370697632431984, 0.10416869819164276, -0.19022507965564728, 0.11423652619123459, 0.09144753962755203, -0.08765758574008942, -0.0032832929864525795, -0.0051014479249715805, 0.19875964522361755, 0.07349056005477905, -0.1031823456287384, -0.10447365045547485, 0.11358538269996643, -0.24666038155555725, -0.05960353836417198, 0.07124857604503632, -0.039664581418037415, -0.20122921466827393, -0.31481748819351196, -0.006801256909966469, 0.41940364241600037, 0.1236235573887825, -0.12495145946741104, 0.12580059468746185, -0.02020396664738655, -0.03004150651395321, 0.11967054009437561, 0.09008713811635971, -0.07470540702342987, 0.09324200451374054, -0.13763070106506348, 0.07720538973808289, 0.19568027555942535, 0.036567769944667816, 0.030284458771348, 0.14119629561901093, -0.03820852190256119, 0.06232285499572754, 0.036639824509620667, 0.07704029232263565, -0.12276224792003632, -0.0035170004703104496, -0.13103705644607544, 0.027697769924998283, -0.01527332328259945, -0.04027168080210686, -0.03659897670149803, 0.03330300375819206, -0.12293602526187897, 0.09043421596288681, -0.019673841074109077, -0.07563626766204834, -0.13991905748844147, 0.014788001775741577, -0.07630413770675659, 0.00017269013915210962, 0.16345393657684326, -0.25710681080818176, 0.19869503378868103, 0.19393865764141083, -0.07422225922346115, 0.19553625583648682, 0.09189949929714203, 0.051557887345552444, -0.0008843056857585907, -0.006250975653529167, -0.1680600494146347, -0.10320111364126205, 0.03232177346944809, -0.08931156992912292, 0.11964476853609085, 0.00814182311296463]
array_1 = []
array_2 = []
for i in range(5):
    array_1.append(random.randint(0,9))
    array_2.append(random.randint(0,9))
V = np.cov(np.array([array_1, array_2]).T)
display(V)

IV = np.linalg.inv(V)
print(distance.mahalanobis(array_1, array_2, IV))

array([[  4.5,   7.5,  -4.5,  -1.5,  -6. ],
       [  7.5,  12.5,  -7.5,  -2.5, -10. ],
       [ -4.5,  -7.5,   4.5,   1.5,   6. ],
       [ -1.5,  -2.5,   1.5,   0.5,   2. ],
       [ -6. , -10. ,   6. ,   2. ,   8. ]])

LinAlgError: Singular matrix

In [61]:
A=np.matrix([[50,95],[95,1.5],[50,95],[95,1.5],[50,95],[95,1.5]])
print(A)
print(type(A))
B=np.linalg.inv(A)
print(B)
print(type(B))

[[50.  95. ]
 [95.   1.5]
 [50.  95. ]
 [95.   1.5]
 [50.  95. ]
 [95.   1.5]]
<class 'numpy.matrix'>


LinAlgError: Last 2 dimensions of the array must be square

In [2]:
import h2o
h2o.init()
h2o.ls()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,1 min 33 secs
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,1 month and 23 days
H2O_cluster_name:,root
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.784 Gb
H2O_cluster_total_cores:,22
H2O_cluster_allowed_cores:,22
H2O_cluster_status:,"locked, healthy"


,key
